<a href="https://colab.research.google.com/github/QuangTrungNguyen/Bitcoin-predictions-and-visualisations-capstone/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CONNECT TO PYDRIVE

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [3]:
# Test if GPU is active
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## IMPORT LIBRARIES

In [4]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import statsmodels.api as sm
from time import time
from scipy import stats
from sklearn.metrics import mean_squared_error
from math import sqrt
from random import randint
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout,Activation
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import EarlyStopping
from keras import initializers
from matplotlib import pyplot
from datetime import datetime
import sklearn.model_selection
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import keras
from sklearn.metrics import mean_squared_error
from math import sqrt
py.init_notebook_mode(connected=True)
%matplotlib inline


Using TensorFlow backend.


##**IMPORT DATASET**

In [5]:
# Read data
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Read the dataset and record the running time
time_1 = time()
data_df = pd.read_csv('DeepLearning.csv', sep = ',', parse_dates=[0], dayfirst = True)
time_2 = time()
print('read data cost '+ str(time_2 - time_1)+' second')

Saving DeepLearning.csv to DeepLearning.csv
User uploaded file "DeepLearning.csv" with length 101502 bytes
read data cost 0.023080110549926758 second


## PREPARE DATA

In [0]:
# Split data into features and labels
n_samples = len(data_df)
X = data_df.iloc[0:n_samples,0:-1].values
Y = data_df.iloc[0:n_samples,-1].values

X_train = X[:-183]
Y_train = Y[:-183]
X_val = X[-183:-93]
Y_val = Y[-183:-93]

# Normalize data for training
scaler1 = MinMaxScaler()
scaler1.fit(X_train,Y_train)
X_train = scaler1.transform(X_train).reshape(len(X_train),X_train.shape[1],1)
X_val = scaler1.transform(X_val).reshape(len(X_val),X_val.shape[1],1)

scaler2 = MinMaxScaler()
scaler2.fit(Y_train.reshape(-1,1))
Y_train = scaler2.transform(Y_train.reshape(-1,1))
Y_val = scaler2.transform(Y_val.reshape(-1,1))

# Normalize data for testing
X_train_2 = X[:-93]
Y_train_2 = Y[:-93]
X_test = X[-90:]
Y_test = Y[-90:]

scaler1 = MinMaxScaler()
scaler1.fit(X_train_2,Y_train_2)
X_train_2 = scaler1.transform(X_train_2).reshape(len(X_train_2),X_train_2.shape[1],1)
X_test = scaler1.transform(X_test).reshape(len(X_test),X_test.shape[1],1)

scaler2 = MinMaxScaler()
scaler2.fit(Y_train_2.reshape(-1,1))
Y_train_2 = scaler2.transform(Y_train_2.reshape(-1,1))
Y_test = scaler2.transform(Y_test.reshape(-1,1))


In [6]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(820, 5, 1)
(820, 1)
(90, 5, 1)
(90, 1)


##**DEFINE FUNCTIONS**

In [0]:
### DISPLAY PLOTLY GRAPHS IN COLAB
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
# The root_mean_squared_error
  def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
    
### PREDICTION USING TEST DATA
def predict(model):
  global X_test, Y_test
  X_test = np.reshape(X_test, (len(X_test), 1, 1))
  # get predictions and then make some transformations to be able to calculate RMSE properly in USD
  prediction = model.predict(X_test) 
  prediction_inverse = scaler.inverse_transform(prediction.reshape(-1, 1)) 
  Y_test_inverse = scaler.inverse_transform(Y_test.reshape(-1, 1)) 
  prediction2_inverse = np.array(prediction_inverse[:,0][:]) 
  Y_test2_inverse = np.array(Y_test_inverse[:,0]) 
  RMSE = sqrt(mean_squared_error(Y_test2_inverse, prediction2_inverse))
  #print('Test RMSE: %.3f' % RMSE)
  return RMSE


### TRAIN LSTM MODEL:
def LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, n_neurons, n_layers, batch_size, epochs, dropout_rate):
  model = Sequential()
  for i in range(0,n_layers):
    if i == 0:
      if n_layers == 1: model.add(LSTM(n_neurons, input_shape=(X_train.shape[1], X_train.shape[2])))
      elif n_layers > 1: model.add(LSTM(n_neurons, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    if i > 0:
      if i < n_layers-1: model.add(LSTM(n_neurons, return_sequences=True))
      elif i == n_layers-1: model.add(LSTM(n_neurons))
  if dropout_rate > 0:
    model.add(Dropout(dropout_rate))
  model.add(Dense(1))

  time_5 = time()
  def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
  # compile and fit the model
  model.compile(loss=root_mean_squared_error, optimizer=optimizer)
  print(model.summary())
  history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=False,
                      validation_data=(X_test, Y_test), verbose = 1)
                      #callbacks = [EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=40, verbose=1)])
  time_6 = time()
  print('Training cost '+ str(time_6 - time_5)+' second')
  return history, model

def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

### PLOT LOSS GRAPHS
def plot_loss(loss, parameter, title, label):

  size = len(loss[0])
  colors = ['blue','red','green','orange','grey']
  data = []
  for i in range(0,len(loss)):
    trace = go.Scatter(
        x = np.arange(0, size, 1),
        y = loss[i],
        mode = 'lines',
        name = label + ' = ' + str(parameter[i]),
        line = dict(color=colors[i], width=2)
    )
    data.append(trace)
  layout = dict(title = 'Test Loss of different ' + title,
                xaxis = dict(title = 'Epoch number'), yaxis = dict(title = 'Loss'))
  fig = dict(data=data, layout=layout)
  py.iplot(fig, filename='training_process')
  
  
  
  # Get best learning rate, loss
  min_loss = []
  for i in range(0,len(loss)):
    min_loss.append(np.min(loss[i]))
    
  best_parameter = parameter[np.argmin(min_loss)]
  best_loss = np.min(min_loss)  

  return best_parameter, best_loss


## HYPER-PARAMETERS TUNING

### OPTIMIZER

#### Adam

In [0]:
time_1 = time()

# Adam
learning_rate = [1e-5,5e-5,1e-4,5e-4,1e-3]   
loss_adam = []
for lr in learning_rate:
  optimizer = Adam(lr=lr, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_adam.append(history.history['val_loss'])  

time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

In [0]:
# Plot loss
configure_plotly_browser_state()
best_learning_rate_adam, best_loss_adam = plot_loss(loss_adam, learning_rate, 'Adam learning rates for LSTM', 'lr')
print('Best learning rate:', str(best_learning_rate_adam))
print('Best loss:', str(best_loss_adam))



#### SGD

In [0]:
time_1 = time()

# RMSprop
learning_rate = [1e-4,5e-4,1e-3,5e-3,1e-2]    # MIGHT NEED TO ADJUST
loss_sgd = []
for lr in learning_rate:
  optimizer = SGD(lr=lr, momentum=0.9, decay=1e-6, nesterov=True)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_sgd.append(history.history['val_loss'])
  
time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

In [0]:
# Plot loss
configure_plotly_browser_state()
best_learning_rate_sgd, best_loss_sgd = plot_loss(loss_sgd, learning_rate, 'SGD learning rates for LSTM', 'lr')
print('Best learning rate:', str(best_learning_rate_sgd))
print('Best loss:', str(best_loss_sgd))


#### RMSprop

In [0]:
time_1 = time()

# RMSprop
learning_rate = [1e-5,5e-5,1e-4,5e-4,1e-3]    # MIGHT NEED TO ADJUST
loss_rmsprop = []
for lr in learning_rate:
  optimizer = RMSprop(lr=lr, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, optimizer, 128, 1, 16, 100, 0)
  loss_rmsprop.append(history.history['val_loss'])
  
time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')


In [0]:
# Plot loss
configure_plotly_browser_state()
best_learning_rate_rmsprop, best_loss_rmsprop = plot_loss(loss_rmsprop, learning_rate, 'RMSprop learning rates for LSTM', 'lr')
print('Best learning rate:', str(best_learning_rate_rmsprop))
print('Best loss:', str(best_loss_rmsprop))



#### Select best optimizer with best learning rate to fine tune other paramters

In [0]:
optimizer = ['Adam','RMSprop','SGD']
best_loss = np.min([best_loss_adam,best_loss_rmsprop,best_loss_sgd])
best_optimizer = optimizer[np.argmin([best_loss_adam,best_loss_rmsprop,best_loss_sgd])]
print('Best optimizer:', best_optimizer)


In [0]:
# Optimizer and learning rates to be used to tune other hyperparameters
best_optimizer = Adam(lr=best_learning_rate_adam , decay=1e-6)

### NUMBER OF NEURONS

In [8]:
time_1 = time()

# Adam
n_neurons = [128,256,512]   
loss_adam = []
for neurons in n_neurons:
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, best_optimizer, neurons, 1, 16, 100, 0)
  loss_adam.append(history.history['val_loss'])  

time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 2s 2ms/step - loss: 0.1149 - val_loss: 0.2970
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1572 - val_loss: 0.3188
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1480 - val_loss: 0.3306
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1413 - val_loss: 0.3272
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.140

In [26]:
# Plot loss
configure_plotly_browser_state()
best_n_neurons, best_loss_neurons = plot_loss(loss_adam, n_neurons, 'number of neurons for LSTM', 'n_neurons')
print('Best number of neurons:', str(best_n_neurons))
print('Best loss: {:0.5f}'.format(best_loss_neurons))



Best number of neurons: 256
Best loss: 0.01357


### NUMBER OF LAYERS

In [11]:
time_1 = time()

# Adam
n_layers = [1,2,3]   
loss_layers = []
for layers in n_layers:
  optimizer = Adam(lr=0.0001, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, best_optimizer, best_n_neurons, layers, 16, 100, 0)
  loss_layers.append(history.history['val_loss'])  

time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 2s 2ms/step - loss: 0.1027 - val_loss: 0.2403
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1792 - val_loss: 0.2944
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1548 - val_loss: 0.3149
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1424 - val_loss: 0.3076
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1

In [24]:
# Plot loss
configure_plotly_browser_state()
best_n_layers, best_loss_layers = plot_loss(loss_layers, n_layers, 'number of layers for LSTM', 'n_layers')
print('Best number of layers:', str(best_n_layers))
print('Best loss: {:0.5f}'.format(best_loss_layers))



Best number of layers: 1
Best loss: 0.01334


### BATCH SIZE

In [15]:
time_1 = time()

# Adam
batch_size = [1,16,32,64]   
loss_batch = []
for batch in batch_size:
  optimizer = Adam(lr=0.0001, decay=1e-6)
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, best_optimizer, best_n_neurons, best_n_layers, batch, 100, 0)
  loss_batch.append(history.history['val_loss'])  

time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 256)               264192    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 16s 19ms/step - loss: 0.0162 - val_loss: 0.0891
Epoch 2/100
819/819 [==============================] - 14s 18ms/step - loss: 0.0393 - val_loss: 0.0825
Epoch 3/100
819/819 [==============================] - 14s 18ms/step - loss: 0.0441 - val_loss: 0.0647
Epoch 4/100
819/819 [==============================] - 14s 18ms/step - loss: 0.0467 - val_loss: 0.0612
Epoch 5/100
819/819 [==============================] - 14s 18ms/step -

In [25]:
# Plot loss
configure_plotly_browser_state()
best_batch_size, best_loss_batch = plot_loss(loss_layers, batch_size, 'batch sizes for LSTM', 'batch_size')
print('Best batch size', str(best_batch_size))
print('Best loss: {:0.5f}'.format(best_loss_batch))


Best batch size 16
Best loss: 0.01317


### DROPOUT RATE

In [28]:
time_1 = time()

# Adam
dropout_rate = [0,0.25,0.5,0.75]   
loss_dropout = []
for dropout in dropout_rate:
  history, model = LSTM_train_model(X_train, Y_train, X_test, Y_test, best_optimizer, best_n_neurons, best_n_layers, best_batch_size, 100, dropout)
  loss_dropout.append(history.history['val_loss'])  

time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 256)               264192    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________
None
Train on 819 samples, validate on 91 samples
Epoch 1/100
819/819 [==============================] - 3s 4ms/step - loss: 0.1013 - val_loss: 0.2404
Epoch 2/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1776 - val_loss: 0.2934
Epoch 3/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1552 - val_loss: 0.3182
Epoch 4/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1407 - val_loss: 0.3097
Epoch 5/100
819/819 [==============================] - 1s 1ms/step - loss: 0.1

In [31]:
# Plot loss
configure_plotly_browser_state()
best_dropout_rate, best_loss_dropout = plot_loss(loss_dropout, dropout_rate, 'dropout rates for LSTM', 'dropout_rate')
print('Best dropout rate', str(best_dropout_rate))
print('Best loss: {:0.5f}'.format(best_loss_dropout))



Best dropout rate 0.75
Best loss: 0.01217


## **FINAL TEST DATA VISUALIZATION**

### Prediction

In [98]:
time_1 = time()

# Best parameters
history, new_model = LSTM_train_model(X_train, Y_train, X_val, Y_val, best_optimizer, best_n_neurons, best_n_layers, best_batch_size, 50, best_dropout_rate)

time_2 = time()
print('Training these models take '+ str(time_2 - time_1)+' second')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_33 (LSTM)               (None, 256)               264192    
_________________________________________________________________
dropout_19 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________
None
Train on 910 samples, validate on 90 samples
Epoch 1/50
910/910 [==============================] - 6s 6ms/step - loss: 0.1313 - val_loss: 0.1714
Epoch 2/50
910/910 [==============================] - 1s 1ms/step - loss: 0.1946 - val_loss: 0.2173
Epoch 3/50
910/910 [==============================] - 1s 1ms/step - loss: 0.1763 - val_loss: 0.2367
Epoch 4/50
910/910 [==============================

In [0]:
# Inverse Y_pred
y_pred = new_model.predict(X_test)
y_true = scaler2.inverse_transform(Y_test)
y_hat = scaler2.inverse_transform(y_pred)


In [100]:
configure_plotly_browser_state()

trace1 = go.Scatter(
    x = np.arange(0, len(y_hat), 1),
    y = y_hat.reshape(-1,),
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(y_true), 1),
    y = y_true.reshape(-1,),
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our LSTM model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating0')

rmse = np.sqrt(mean_squared_error(y_true.reshape(-1,1),y_hat.reshape(-1,1)))

print('RMSE: {:0.5f}'.format(rmse))



RMSE: 297.75032


In [0]:
# Save predictions to csv file
from pandas import DataFrame
y_p = {'Y_pred': y_hat.reshape(-1,)}
y_p = DataFrame(y_p)
y_p.to_csv('LSTM.csv', index=False)
files.download('LSTM.csv')